In [ ]:
import torch                                            # Framework
import torch.nn as nn                                   # Layer architecture
import torch.optim as optim                             # Optimizers
from torch.optim import lr_scheduler                    # Learning rate adjuster
import torch.backends.cudnn as cudnn                    # CUDA interface
import numpy as np                                      # yep
import torchvision                                      # CV torch
from torchvision import datasets, models, transforms    # yep
import matplotlib.pyplot as plt                         # yep
import time                                             # yep
import os                                               # yep
from PIL import Image                                   # yep
from tempfile import TemporaryDirectory                 # Automatic temporary folders

cudnn.benchmark = True          # Auto setup
plt.ion()                       # interactive mode

```Load data```

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([                       # Single pipeline of transforms
        transforms.RandomResizedCrop(224),              # Random crop every access to an image (e.g. epoch)
        transforms.RandomHorizontalFlip(),              # Random flip every access to an image (e.g. epoch)
        transforms.ToTensor(),                          # Image to pytorch tensor with range of [0-1]
        transforms.Normalize([0.485, 0.456, 0.406],     # Mean color
                             [0.229, 0.224, 0.225])     # std of colors
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),                         # Resize smaller side of an image to 256
        transforms.CenterCrop(224),                     # Crop center of a 224 side
        transforms.ToTensor(),                          
        transforms.Normalize([0.485, 0.456, 0.406],     
                             [0.229, 0.224, 0.225])     
    ]),
}

data_dir = 'data/hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")